In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from supabase import create_client
from dotenv import load_dotenv

load_dotenv()

supabase = create_client(
    os.getenv("SUPABASE_URL"),
    os.getenv("SUPABASE_KEY")
)

response = supabase.table("churn_data").select("*").execute()
df = pd.DataFrame(response.data)

churn_percentage = (
    df["churn"]
    .value_counts(normalize=True) * 100
).round(2)

avg_monthly_contract = (
    df.groupby("contract")["monthlycharges"]
    .mean()
    .round(2)
)

tenure_counts = df["tenure_group"].value_counts()
internet_distribution = df["internetservice"].value_counts()

churn_tenure_pivot = pd.pivot_table(
    df,
    index="tenure_group",
    columns="churn",
    values="customerid",
    aggfunc="count",
    fill_value=0
)

churn_by_charge_segment = (
    df.groupby("monthlycharges_group")["churn"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
    .round(2) * 100
)

graphs_path = r"C:\Users\srira\OneDrive\Desktop\ETL pipeline 2\data\processed"

plt.figure(figsize=(8,5))
churn_by_charge_segment.plot(kind="bar")
plt.tight_layout()
plt.savefig(os.path.join(graphs_path, "churn_by_monthly_charge_segment.png"))
plt.close()

plt.figure()
df["totalcharges"].hist(bins=30)
plt.tight_layout()
plt.savefig(os.path.join(graphs_path, "total_charges_distribution.png"))
plt.close()

plt.figure()
df["contract"].value_counts().plot(kind="bar")
plt.tight_layout()
plt.savefig(os.path.join(graphs_path, "contract_type_distribution.png"))
plt.close()

summary_path = os.path.join(graphs_path, "analysis_summary.csv")

summary_df = pd.DataFrame({
    "Churn_Percentage": churn_percentage
})

summary_df.to_csv(summary_path)

summary_df


,Churn_Percentage
churn,
False,74.4
True,25.6


<Figure size 800x500 with 0 Axes>